![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/ER_SNOMED_ES.ipynb)

# Colab Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.8
Spark NLP_JSL Version : 4.2.8


# Checking in an inference pipeline

In [4]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

word_embeddings = RoBertaEmbeddings.pretrained("roberta_base_biomedical", "es")\
    .setInputCols(["document", "token"])\
    .setOutputCol("roberta_embeddings")

ner = MedicalNerModel.pretrained("roberta_ner_diag_proc","es","clinical/models")\
    .setInputCols("sentence","token","roberta_embeddings")\
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

snomed_prepro_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        ner,
        ner_converter
    ])

empty = spark.createDataFrame([['']]).toDF("text")

p_model = snomed_prepro_pipeline.fit(empty)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
roberta_base_biomedical download started this may take some time.
Approximate size to download 288 MB
[OK!]
roberta_ner_diag_proc download started this may take some time.
[OK!]


In [5]:
import pandas as pd

test_sentence = 'Mujer de 28 años con antecedentes de diabetes mellitus gestacional diagnosticada ocho años antes de la presentación y posterior diabetes mellitus tipo dos (DM2), un episodio previo de pancreatitis inducida por HTG tres años antes de la presentación, asociado con una hepatitis aguda, y obesidad con un índice de masa corporal (IMC) de 33,5 kg / m2, que se presentó con antecedentes de una semana de poliuria, polidipsia, falta de apetito y vómitos. Dos semanas antes de la presentación, fue tratada con un ciclo de cinco días de amoxicilina por una infección del tracto respiratorio. Estaba tomando metformina, glipizida y dapagliflozina para la DM2 y atorvastatina y gemfibrozil para la HTG. Había estado tomando dapagliflozina durante seis meses en el momento de la presentación. El examen físico al momento de la presentación fue significativo para la mucosa oral seca; significativamente, su examen abdominal fue benigno sin dolor a la palpación, protección o rigidez. Los hallazgos de laboratorio pertinentes al ingreso fueron: glucosa sérica 111 mg / dl, bicarbonato 18 mmol / l, anión gap 20, creatinina 0,4 mg / dl, triglicéridos 508 mg / dl, colesterol total 122 mg / dl, hemoglobina glucosilada (HbA1c) 10%. y pH venoso 7,27. La lipasa sérica fue normal a 43 U / L. Los niveles séricos de acetona no pudieron evaluarse ya que las muestras de sangre se mantuvieron hemolizadas debido a una lipemia significativa. La paciente ingresó inicialmente por cetosis por inanición, ya que refirió una ingesta oral deficiente durante los tres días previos a la admisión. Sin embargo, la química sérica obtenida seis horas después de la presentación reveló que su glucosa era de 186 mg / dL, la brecha aniónica todavía estaba elevada a 21, el bicarbonato sérico era de 16 mmol / L, el nivel de triglicéridos alcanzó un máximo de 2050 mg / dL y la lipasa fue de 52 U / L. Se obtuvo el nivel de β-hidroxibutirato y se encontró que estaba elevado a 5,29 mmol / L; la muestra original se centrifugó y la capa de quilomicrones se eliminó antes del análisis debido a la interferencia de la turbidez causada por la lipemia nuevamente. El paciente fue tratado con un goteo de insulina para euDKA y HTG con una reducción de la brecha aniónica a 13 y triglicéridos a 1400 mg / dL, dentro de las 24 horas. Se pensó que su euDKA fue precipitada por su infección del tracto respiratorio en el contexto del uso del inhibidor de SGLT2. La paciente fue atendida por el servicio de endocrinología y fue dada de alta con 40 unidades de insulina glargina por la noche, 12 unidades de insulina lispro con las comidas y metformina 1000 mg dos veces al día. Se determinó que todos los inhibidores de SGLT2 deben suspenderse indefinidamente. Tuvo un seguimiento estrecho con endocrinología post alta.'
res = p_model.transform(spark.createDataFrame(pd.DataFrame({'text': [test_sentence]})))

In [6]:
c2doc = Chunk2Doc() \
    .setInputCols("ner_chunk") \
    .setOutputCol("sentence")

chunk_tokenizer = Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

chunk_word_embeddings = RoBertaEmbeddings.pretrained("roberta_base_biomedical", "es")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("ner_chunk_word_embeddings")

chunk_embeddings = SentenceEmbeddings() \
    .setInputCols(["sentence", "ner_chunk_word_embeddings"]) \
    .setOutputCol("ner_chunk_embeddings") \
    .setPoolingStrategy("AVERAGE")

er = SentenceEntityResolverModel.pretrained("robertaresolve_snomed", "es", "clinical/models")\
    .setInputCols(["ner_chunk_embeddings"]) \
    .setOutputCol("snomed_code") \
    .setDistanceFunction("EUCLIDEAN")

snomed_resolve_pipeline = Pipeline(
    stages = [
        c2doc,
        chunk_tokenizer,
        chunk_word_embeddings,
        chunk_embeddings,
        er
        ])


roberta_base_biomedical download started this may take some time.
Approximate size to download 288 MB
[OK!]
robertaresolve_snomed download started this may take some time.
[OK!]


In [7]:
p_infer_model = snomed_resolve_pipeline.fit(res.select('ner_chunk'))

In [8]:
final_res = p_infer_model.transform(res.select('ner_chunk'))

In [9]:
import pandas as pd

chunk = 'ner_chunk'
output_col='snomed_code'

pd.set_option('display.max_colwidth', 0)

df = final_res.select(F.explode(F.arrays_zip(final_res[chunk].begin, 
                                             final_res[chunk].end, 
                                             final_res[chunk].result, 
                                             final_res[chunk].metadata, 
                                             final_res[output_col].result, 
                                             final_res[output_col].metadata)).alias("cols")) \
              .select(F.expr("cols['0']").alias("begin"),
                      F.expr("cols['1']").alias("end"),
                      F.expr("cols['3']['sentence']").alias("sent_id"),
                      F.expr("cols['2']").alias("ner_chunk"),
                      F.expr("cols['3']['entity']").alias("entity"), 
                      F.expr("cols['4']").alias("snomed_code"),
                      F.expr("cols['5']['all_k_resolutions']").alias("snomed_texts"),                                     
                      ).toPandas()

In [10]:
df

,begin,end,sent_id,ner_chunk,entity,snomed_code,snomed_texts
0,37,65,0,diabetes mellitus gestacional,DIAGNOSTICO,11687002,diabetes mellitus gestacional:::diabetes mellitus gestacional:::diabetes mellitus inestable:::diabetes mellitus inestable:::diabetes mellitus materna:::diabetes mellitus medicamentosa:::diabetes mellitus postrasplante:::diabetes mellitus atípica:::diabetes mellitus gestacional posparto:::diabetes mellitus lipoatrófica:::diabetes mellitus neonatal:::diabetes mellitus neonatal permanente:::diabetes mellitus con gangrena:::diabetes mellitus gestacional no controlada:::diabetes mellitus autosómica dominante:::diabetes mellitus tipo I inestable:::diabetes mellitus con cetoacidosis:::[X]otra diabetes mellitus especificada:::diabetes mellitus con complicaciones
1,128,153,0,diabetes mellitus tipo dos,DIAGNOSTICO,11687002,diabetes mellitus gestacional:::diabetes mellitus gestacional:::diabetes mellitus inestable:::diabetes mellitus inestable:::diabetes mellitus materna:::diabetes mellitus medicamentosa:::diabetes mellitus postrasplante:::diabetes mellitus atípica:::diabetes mellitus gestacional posparto:::diabetes mellitus lipoatrófica:::diabetes mellitus neonatal:::diabetes mellitus neonatal permanente:::diabetes mellitus con gangrena:::diabetes mellitus gestacional no controlada:::diabetes mellitus autosómica dominante:::diabetes mellitus tipo I inestable:::diabetes mellitus con cetoacidosis:::[X]otra diabetes mellitus especificada:::diabetes mellitus con complicaciones
2,184,195,0,pancreatitis,DIAGNOSTICO,11687002,diabetes mellitus gestacional:::diabetes mellitus gestacional:::diabetes mellitus inestable:::diabetes mellitus inestable:::diabetes mellitus materna:::diabetes mellitus medicamentosa:::diabetes mellitus postrasplante:::diabetes mellitus atípica:::diabetes mellitus gestacional posparto:::diabetes mellitus lipoatrófica:::diabetes mellitus neonatal:::diabetes mellitus neonatal permanente:::diabetes mellitus con gangrena:::diabetes mellitus gestacional no controlada:::diabetes mellitus autosómica dominante:::diabetes mellitus tipo I inestable:::diabetes mellitus con cetoacidosis:::[X]otra diabetes mellitus especificada:::diabetes mellitus con complicaciones
3,267,282,0,"hepatitis aguda,",DIAGNOSTICO,11687002,diabetes mellitus gestacional:::diabetes mellitus gestacional:::diabetes mellitus inestable:::diabetes mellitus inestable:::diabetes mellitus materna:::diabetes mellitus medicamentosa:::diabetes mellitus postrasplante:::diabetes mellitus atípica:::diabetes mellitus gestacional posparto:::diabetes mellitus lipoatrófica:::diabetes mellitus neonatal:::diabetes mellitus neonatal permanente:::diabetes mellitus con gangrena:::diabetes mellitus gestacional no controlada:::diabetes mellitus autosómica dominante:::diabetes mellitus tipo I inestable:::diabetes mellitus con cetoacidosis:::[X]otra diabetes mellitus especificada:::diabetes mellitus con complicaciones
4,286,293,0,obesidad,DIAGNOSTICO,11687002,diabetes mellitus gestacional:::diabetes mellitus gestacional:::diabetes mellitus inestable:::diabetes mellitus inestable:::diabetes mellitus materna:::diabetes mellitus medicamentosa:::diabetes mellitus postrasplante:::diabetes mellitus atípica:::diabetes mellitus gestacional posparto:::diabetes mellitus lipoatrófica:::diabetes mellitus neonatal:::diabetes mellitus neonatal permanente:::diabetes mellitus con gangrena:::diabetes mellitus gestacional no controlada:::diabetes mellitus autosómica dominante:::diabetes mellitus tipo I inestable:::diabetes mellitus con cetoacidosis:::[X]otra diabetes mellitus especificada:::diabetes mellitus con complicaciones
5,302,324,0,índice de masa corporal,DIAGNOSTICO,11687002,diabetes mellitus gestacional:::diabetes mellitus gestacional:::diabetes mellitus inestable:::diabetes mellitus inestable:::diabetes mellitus materna:::diabetes mellitus medicamentosa:::diabetes mellitus postrasplante:::diabetes mellitus atípica:::diabetes mellitus gestacional posparto:::diabetes mellitus